In [1]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 380
batch_size = 100
verbose = 1
num_classes = 8

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

d:\users\damon\documents\sxsw18\deadsimplespeechrecognizer\env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Saving vectors of label - 'toms': 100%|███████████████████████████████████████████████| 24/24 [00:00<00:00, 245.45it/s]


In [2]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

import time

def test(directory, model):
    labels = os.listdir(directory)
    for l in labels:
        files = os.listdir(os.path.join(directory, l))
        for f in files:
            path = os.path.join(directory, l, f)
            start = time.time()
            print (l, predict(path, model), time.time()-start, "secs")
        
    

# Building The Model Then Training it

In [3]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

# export model
model.save('saved_model')


Train on 109 samples, validate on 73 samples
Epoch 1/380
109/109 [==============================] - ETA: 0s - loss: 8.2533 - acc: 0.090 - 5s 43ms/step - loss: 7.9339 - acc: 0.1009 - val_loss: 3.3274 - val_acc: 0.0959
Epoch 2/380
109/109 [==============================] - ETA: 0s - loss: 4.5966 - acc: 0.130 - 0s 221us/step - loss: 4.5194 - acc: 0.1284 - val_loss: 2.0168 - val_acc: 0.2740
Epoch 3/380
109/109 [==============================] - ETA: 0s - loss: 2.4491 - acc: 0.240 - 0s 239us/step - loss: 2.3569 - acc: 0.2661 - val_loss: 1.9536 - val_acc: 0.3014
Epoch 4/380
109/109 [==============================] - ETA: 0s - loss: 2.2194 - acc: 0.290 - 0s 230us/step - loss: 2.1554 - acc: 0.2936 - val_loss: 1.4893 - val_acc: 0.5479
Epoch 5/380
109/109 [==============================] - ETA: 0s - loss: 1.6519 - acc: 0.450 - 0s 267us/step - loss: 1.6852 - acc: 0.4220 - val_loss: 1.6468 - val_acc: 0.3562
Epoch 6/380
109/109 [==============================] - ETA: 0s - loss: 1.8168 - acc: 0.370 

Epoch 48/380
109/109 [==============================] - ETA: 0s - loss: 0.2227 - acc: 0.910 - 0s 225us/step - loss: 0.2255 - acc: 0.9083 - val_loss: 0.2559 - val_acc: 0.9178
Epoch 49/380
109/109 [==============================] - ETA: 0s - loss: 0.2305 - acc: 0.890 - 0s 225us/step - loss: 0.2245 - acc: 0.8991 - val_loss: 0.2644 - val_acc: 0.9315
Epoch 50/380
109/109 [==============================] - ETA: 0s - loss: 0.1505 - acc: 0.950 - 0s 225us/step - loss: 0.1426 - acc: 0.9541 - val_loss: 0.2427 - val_acc: 0.9315
Epoch 51/380
109/109 [==============================] - ETA: 0s - loss: 0.2173 - acc: 0.930 - 0s 239us/step - loss: 0.2462 - acc: 0.9266 - val_loss: 0.8543 - val_acc: 0.7260
Epoch 52/380
109/109 [==============================] - ETA: 0s - loss: 0.4688 - acc: 0.820 - 0s 216us/step - loss: 0.4328 - acc: 0.8349 - val_loss: 0.3158 - val_acc: 0.9178
Epoch 53/380
109/109 [==============================] - ETA: 0s - loss: 0.2164 - acc: 0.930 - 0s 235us/step - loss: 0.2068 - acc: 

109/109 [==============================] - ETA: 0s - loss: 0.0382 - acc: 0.990 - 0s 221us/step - loss: 0.0369 - acc: 0.9908 - val_loss: 0.2602 - val_acc: 0.9315
Epoch 96/380
109/109 [==============================] - ETA: 0s - loss: 0.0587 - acc: 0.970 - 0s 225us/step - loss: 0.0543 - acc: 0.9725 - val_loss: 0.2324 - val_acc: 0.9589
Epoch 97/380
109/109 [==============================] - ETA: 0s - loss: 0.0726 - acc: 0.970 - 0s 216us/step - loss: 0.0691 - acc: 0.9725 - val_loss: 0.2107 - val_acc: 0.9452
Epoch 98/380
109/109 [==============================] - ETA: 0s - loss: 0.0413 - acc: 0.980 - 0s 239us/step - loss: 0.0539 - acc: 0.9725 - val_loss: 0.2487 - val_acc: 0.9452
Epoch 99/380
109/109 [==============================] - ETA: 0s - loss: 0.0711 - acc: 0.970 - 0s 225us/step - loss: 0.0653 - acc: 0.9725 - val_loss: 0.2264 - val_acc: 0.9452
Epoch 100/380
109/109 [==============================] - ETA: 0s - loss: 0.0156 - acc: 1.000 - 0s 221us/step - loss: 0.0191 - acc: 1.0000 - val

Epoch 142/380
109/109 [==============================] - ETA: 0s - loss: 0.0165 - acc: 1.000 - 0s 221us/step - loss: 0.0170 - acc: 1.0000 - val_loss: 0.2834 - val_acc: 0.9452
Epoch 143/380
109/109 [==============================] - ETA: 0s - loss: 0.0106 - acc: 1.000 - 0s 225us/step - loss: 0.0100 - acc: 1.0000 - val_loss: 0.2744 - val_acc: 0.9452
Epoch 144/380
109/109 [==============================] - ETA: 0s - loss: 0.0233 - acc: 1.000 - 0s 267us/step - loss: 0.0215 - acc: 1.0000 - val_loss: 0.2648 - val_acc: 0.9589
Epoch 145/380
109/109 [==============================] - ETA: 0s - loss: 0.0344 - acc: 0.980 - 0s 226us/step - loss: 0.0326 - acc: 0.9817 - val_loss: 0.2875 - val_acc: 0.9452
Epoch 146/380
109/109 [==============================] - ETA: 0s - loss: 0.0175 - acc: 1.000 - 0s 225us/step - loss: 0.0214 - acc: 1.0000 - val_loss: 0.3466 - val_acc: 0.9178
Epoch 147/380
109/109 [==============================] - ETA: 0s - loss: 0.0804 - acc: 0.980 - 0s 272us/step - loss: 0.0743 -

Epoch 189/380
109/109 [==============================] - ETA: 0s - loss: 0.0094 - acc: 1.000 - 0s 253us/step - loss: 0.0087 - acc: 1.0000 - val_loss: 0.3496 - val_acc: 0.9315
Epoch 190/380
109/109 [==============================] - ETA: 0s - loss: 0.0081 - acc: 1.000 - 0s 225us/step - loss: 0.0075 - acc: 1.0000 - val_loss: 0.3358 - val_acc: 0.9315
Epoch 191/380
109/109 [==============================] - ETA: 0s - loss: 0.0317 - acc: 0.990 - 0s 230us/step - loss: 0.0297 - acc: 0.9908 - val_loss: 0.3402 - val_acc: 0.9452
Epoch 192/380
109/109 [==============================] - ETA: 0s - loss: 0.0138 - acc: 1.000 - 0s 271us/step - loss: 0.0137 - acc: 1.0000 - val_loss: 0.3378 - val_acc: 0.9452
Epoch 193/380
109/109 [==============================] - ETA: 0s - loss: 0.0070 - acc: 1.000 - 0s 230us/step - loss: 0.0064 - acc: 1.0000 - val_loss: 0.3313 - val_acc: 0.9452
Epoch 194/380
109/109 [==============================] - ETA: 0s - loss: 0.0061 - acc: 1.000 - 0s 244us/step - loss: 0.0067 -

Epoch 236/380
109/109 [==============================] - ETA: 0s - loss: 0.0362 - acc: 0.980 - 0s 239us/step - loss: 0.0336 - acc: 0.9817 - val_loss: 0.2922 - val_acc: 0.9452
Epoch 237/380
109/109 [==============================] - ETA: 0s - loss: 0.0073 - acc: 1.000 - 0s 211us/step - loss: 0.0067 - acc: 1.0000 - val_loss: 0.2900 - val_acc: 0.9452
Epoch 238/380
109/109 [==============================] - ETA: 0s - loss: 0.0107 - acc: 0.990 - 0s 244us/step - loss: 0.0100 - acc: 0.9908 - val_loss: 0.2859 - val_acc: 0.9452
Epoch 239/380
109/109 [==============================] - ETA: 0s - loss: 7.3150e-04 - acc: 1.000 - 0s 248us/step - loss: 7.2799e-04 - acc: 1.0000 - val_loss: 0.2869 - val_acc: 0.9452
Epoch 240/380
109/109 [==============================] - ETA: 0s - loss: 0.0144 - acc: 0.990 - 0s 221us/step - loss: 0.0132 - acc: 0.9908 - val_loss: 0.2836 - val_acc: 0.9452
Epoch 241/380
109/109 [==============================] - ETA: 0s - loss: 0.0745 - acc: 0.980 - 0s 225us/step - loss: 

Epoch 283/380
109/109 [==============================] - ETA: 0s - loss: 0.0010 - acc: 1.000 - 0s 212us/step - loss: 0.0047 - acc: 1.0000 - val_loss: 0.3946 - val_acc: 0.9452
Epoch 284/380
109/109 [==============================] - ETA: 0s - loss: 5.4298e-04 - acc: 1.000 - 0s 244us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.3737 - val_acc: 0.9452
Epoch 285/380
109/109 [==============================] - ETA: 0s - loss: 0.0035 - acc: 1.000 - 0s 253us/step - loss: 0.0034 - acc: 1.0000 - val_loss: 0.3753 - val_acc: 0.9452
Epoch 286/380
109/109 [==============================] - ETA: 0s - loss: 0.0175 - acc: 0.990 - 0s 239us/step - loss: 0.0269 - acc: 0.9817 - val_loss: 0.3033 - val_acc: 0.9452
Epoch 287/380
109/109 [==============================] - ETA: 0s - loss: 0.0039 - acc: 1.000 - 0s 239us/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.3342 - val_acc: 0.9452
Epoch 288/380
109/109 [==============================] - ETA: 0s - loss: 0.0059 - acc: 1.000 - 0s 230us/step - loss: 0.00

Epoch 330/380
109/109 [==============================] - ETA: 0s - loss: 0.0381 - acc: 0.990 - 0s 235us/step - loss: 0.0351 - acc: 0.9908 - val_loss: 0.3884 - val_acc: 0.9452
Epoch 331/380
109/109 [==============================] - ETA: 0s - loss: 0.0229 - acc: 0.990 - 0s 216us/step - loss: 0.0210 - acc: 0.9908 - val_loss: 0.4025 - val_acc: 0.9452
Epoch 332/380
109/109 [==============================] - ETA: 0s - loss: 0.0156 - acc: 1.000 - 0s 202us/step - loss: 0.0144 - acc: 1.0000 - val_loss: 0.4109 - val_acc: 0.9452
Epoch 333/380
109/109 [==============================] - ETA: 0s - loss: 0.0174 - acc: 0.990 - 0s 221us/step - loss: 0.0160 - acc: 0.9908 - val_loss: 0.3959 - val_acc: 0.9452
Epoch 334/380
109/109 [==============================] - ETA: 0s - loss: 0.0040 - acc: 1.000 - 0s 202us/step - loss: 0.0039 - acc: 1.0000 - val_loss: 0.4138 - val_acc: 0.9452
Epoch 335/380
109/109 [==============================] - ETA: 0s - loss: 0.0032 - acc: 1.000 - 0s 235us/step - loss: 0.0031 -

109/109 [==============================] - ETA: 0s - loss: 0.0025 - acc: 1.000 - 0s 235us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.4430 - val_acc: 0.9452
Epoch 377/380
109/109 [==============================] - ETA: 0s - loss: 0.0054 - acc: 1.000 - 0s 216us/step - loss: 0.0059 - acc: 1.0000 - val_loss: 0.4424 - val_acc: 0.9452
Epoch 378/380
109/109 [==============================] - ETA: 0s - loss: 0.0129 - acc: 0.990 - 0s 235us/step - loss: 0.0118 - acc: 0.9908 - val_loss: 0.4508 - val_acc: 0.9452
Epoch 379/380
109/109 [==============================] - ETA: 0s - loss: 0.0469 - acc: 0.990 - 0s 202us/step - loss: 0.0431 - acc: 0.9908 - val_loss: 0.4757 - val_acc: 0.9452
Epoch 380/380
109/109 [==============================] - ETA: 0s - loss: 0.0041 - acc: 1.000 - 0s 239us/step - loss: 0.0039 - acc: 1.0000 - val_loss: 0.4600 - val_acc: 0.9452


In [6]:
test('damon-trainingdata-clean', model)

bass-drum bass-drum 0.008505582809448242 secs
bass-drum bass-drum 0.009519338607788086 secs
bass-drum bass-drum 0.010027170181274414 secs
bass-drum bass-drum 0.009024381637573242 secs
bass-drum bass-drum 0.00852203369140625 secs
bass-drum bass-drum 0.010025262832641602 secs
bass-drum bass-drum 0.00852203369140625 secs
bass-drum bass-drum 0.009529352188110352 secs
bass-drum bass-drum 0.010026931762695312 secs
bass-drum bass-drum 0.010025262832641602 secs
bass-drum bass-drum 0.009523868560791016 secs
bass-drum bass-drum 0.00952601432800293 secs
bass-drum bass-drum 0.009523630142211914 secs
bass-drum bass-drum 0.008530139923095703 secs
bass-drum bass-drum 0.009524106979370117 secs
bass-drum bass-drum 0.010024309158325195 secs
bass-drum bass-drum 0.008521556854248047 secs
bass-drum bass-drum 0.009022951126098633 secs
bass-drum bass-drum 0.009516716003417969 secs
bass-drum bass-drum 0.008522272109985352 secs
cowbell cowbell 0.00952291488647461 secs
cowbell cowbell 0.010026693344116211 secs


In [7]:
json_string = model.to_json()
f = open('model.json', 'w')
f.write(json_string)
f.close()

In [6]:
json_string

'{"class_name": "Sequential", "config": [{"class_name": "Conv2D", "config": {"name": "conv2d_4", "trainable": true, "batch_input_shape": [null, 20, 11, 1], "dtype": "float32", "filters": 32, "kernel_size": [2, 2], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Conv2D", "config": {"name": "conv2d_5", "trainable": true, "filters": 48, "kernel_size": [2, 2], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "conf

In [8]:
model.save_weights('model_weights.h5')